Use sql in postgresql to reorder frontpage stories efficiently.

In [7]:
import sqlparse
from django.db.models import Func, Value, BooleanField
from django.db.models.functions import Rank, DenseRank, RowNumber, Cast
from django.db.models import F, Window, DurationField, ExpressionWrapper, DateTimeField, IntegerField, FloatField
from django.db.models import OuterRef, Subquery, Sum

class Days(Func):
    """Cast float field to Interval in days"""
    output_field=DurationField()
    template="to_char(%(expressions)s, 'S9990.0999 \"days\"')::interval"
    
class Epoch(Func):
    """Get epoch timestamp from date time field """
    output_field=FloatField()
    template="extract(epoch from %(expressions)s ) / 3600"

class English(Func):
    """Is the language english"""
    output_field=BooleanField()
    template="%(expressions)s = 'en'"
    

adjusted_publication_time=F('story__publication_date') + Days('priority')
adjusted_ranking = Window(expression=RowNumber(), partition_by=[English('story__language')], order_by=adjusted_publication_time.desc(nulls_last=True))
qry = FrontpageStory.objects.published().annotate(ranking=adjusted_ranking).values('id', 'ranking')
raw_query = sqlparse.format(str(qry.query), reindent=True, keyword_case='upper')
print(raw_query)

update_ordering = f"""
WITH ordered_frontpage AS ( {raw_query} )
UPDATE "frontpage_frontpagestory" SET "order" = "ordered_frontpage"."ranking"
FROM "ordered_frontpage"
WHERE "frontpage_frontpagestory"."id" = "ordered_frontpage"."id";
"""
from django.db import connection
with connection.cursor() as cursor:
   cursor.execute(update_ordering) 
print(update_ordering)

SELECT "frontpage_frontpagestory"."id",
       ROW_NUMBER() OVER (PARTITION BY "stories_story"."language" = 'en'
                          ORDER BY ("stories_story"."publication_date" + to_char("frontpage_frontpagestory"."priority", 'S9990.0999 "days"')::interval) DESC NULLS LAST) AS "ranking"
FROM "frontpage_frontpagestory"
INNER JOIN "stories_story" ON ("frontpage_frontpagestory"."story_id" = "stories_story"."id")
WHERE ("frontpage_frontpagestory"."published" = TRUE
       AND "stories_story"."publication_status" IN (10,
                                                    11))

WITH ordered_frontpage AS ( SELECT "frontpage_frontpagestory"."id",
       ROW_NUMBER() OVER (PARTITION BY "stories_story"."language" = 'en'
                          ORDER BY ("stories_story"."publication_date" + to_char("frontpage_frontpagestory"."priority", 'S9990.0999 "days"')::interval) DESC NULLS LAST) AS "ranking"
FROM "frontpage_frontpagestory"
INNER JOIN "stories_story" ON ("frontpage_frontpagestory".

In [4]:
FrontpageStory.objects.update(order=0)
FrontpageStory.objects.first().save()
#FrontpageStory.objects.reorder()
list(FrontpageStory.objects.all().values('order', 'story__language', 'pk', 'headline', 'story__publication_date').order_by('order'))[:10]

[{'order': 1,
  'story__language': 'nb',
  'pk': 87396,
  'headline': 'Studenttopp føler seg misbrukt i UiO-reklame',
  'story__publication_date': datetime.datetime(2018, 5, 28, 13, 54, tzinfo=<UTC>)},
 {'order': 1,
  'story__language': 'en',
  'pk': 87388,
  'headline': 'Are you feeling alright, guys?',
  'story__publication_date': datetime.datetime(2018, 4, 11, 13, 14, 15, 884771, tzinfo=<UTC>)},
 {'order': 2,
  'story__language': 'en',
  'pk': 87387,
  'headline': 'Business as usual for students after Facebook data breach',
  'story__publication_date': datetime.datetime(2018, 4, 12, 12, 36, 42, 492117, tzinfo=<UTC>)},
 {'order': 2,
  'story__language': 'nb',
  'pk': 87437,
  'headline': 'Sitt stille og hold kjeft – her tenkes det!',
  'story__publication_date': datetime.datetime(2018, 4, 9, 11, 39, tzinfo=<UTC>)},
 {'order': 3,
  'story__language': 'nb',
  'pk': 87395,
  'headline': 'Droppet ut av BI. Nå arrangerer de «stjerne»-konferanse',
  'story__publication_date': datetime.date

In [ ]:
qry = FrontpageStory.objects.annotate(ranking=adjusted_ranking).values('id', 'ranking', 'order')
raw_query = sqlparse.format(str(qry.query), reindent=True, keyword_case='upper')
print(raw_query)
list(qry[:20])

query from [stack overflow](https://stackoverflow.com/a/4359354/1977847)

```sql
WITH v_table_name AS
(
    SELECT row_number() over (partition by col2 order by col3) AS rn, primary_key
    FROM table_name
) 
UPDATE table_name set table_name.col1 = v_table_name.rn
FROM v_table_name
WHERE table_name.primary_key = v_table_name.primary_key;  
```